Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [25]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from copy import copy
import math

import numpy as np
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse

In [26]:


def fitness2(state,problem_size):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(problem_size)]),
        )
    )
    return valid, -cost

fitness = fitness2

In [27]:
def tweak(state,problem_size):
    new_state = copy(state)
    index = randint(0, problem_size - 1)
    new_state[index] = not new_state[index]
    return new_state

### Simulated Annealing
I made a modification to the algorithm so that when you calculate the probability you give more importance to the first term returned by fitness, also save on one hand the last state that has a valid solution so that if the algorithm ends with an invalid solution it returns the last valid

In [28]:
def simulated_annealing(initial_state,problem_size):
    cooling_rate=0.97
    current_state=initial_state
    cost=fitness(current_state,problem_size)
    best_solution=current_state
    best_cost=cost
    temperature=cooling_rate
    last_valid=current_state
    last_cost=fitness(current_state,problem_size)
    cnt=1
    while temperature > 0.0001 :  
            new_state = tweak(current_state,problem_size)
            score_new=fitness(new_state,problem_size)
            score_curr=fitness(current_state,problem_size)
            cnt+=2
            if  score_new>score_curr or random() < math.exp( -((2*score_curr[0]+score_curr[1])-(2*score_new[0]+score_new[1]))/ temperature):
                current_state = new_state
                current_energy =score_new
                if(score_new[0]==problem_size):
                     last_valid=current_state
                     last_cost=score_new
                
                
                if score_new>best_cost:
                    best_solution = current_state
                    best_cost = score_new
            
            temperature *= cooling_rate
    if(best_cost<=last_cost):
         best_cost=last_cost
         best_solution=last_valid

    return best_cost,cnt
    

In [29]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [30]:
def tabu_search(initial_state,problem_size):
    current_state = initial_state
    best_state = current_state
    best_value = fitness(current_state,problem_size)
    tabu_list = []
    cnt=0
    for _ in range(100):
        neighbors = [tweak(current_state,problem_size) for _ in range(100)]  
        best_neighbor = neighbors[0]

        for neighbor in neighbors:
            if  neighbor not in tabu_list :
                best_neighbor_value = fitness(best_neighbor,problem_size)
                neighbor_value = fitness(neighbor,problem_size)
                cnt+=2

                if neighbor_value>best_neighbor_value:
                     best_neighbor = neighbor
                     best_neighbor_value = neighbor_value

        current_state = best_neighbor
        current_value = best_neighbor_value

        if current_value > best_value:
            best_state = current_state
            best_value = current_value

        tabu_list.append(best_neighbor)
        if len(tabu_list) > 100:
            tabu_list.pop(0)
    

    return best_value,cnt



In [31]:
def iterated_local_search(current_state,problem_size):
    steady_state_cont = 0
    steady_state_limit = 10
    cnt=0
    for iterations in range(10000):
        new_state = tweak(current_state,problem_size)
        if fitness(new_state,problem_size) >= fitness(current_state,problem_size):
            steady_state_cont = 0
            cnt+=2
            current_state = new_state
            
        else:
            steady_state_cont+=1
            if steady_state_cont == steady_state_limit :
                tempstate = [choice([True, False]) for _ in range(problem_size)]
                if fitness(tempstate,problem_size) > fitness(current_state,problem_size):
                    current_state=tempstate
                steady_state_cont=0
    return fitness(current_state,problem_size),cnt

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [32]:
num_point=[100,1000,5000]
density=[0.3,0.7]
for i in density:
    for j in num_point:

        current_state = [choice([False, False, False, False, False, False]) for _ in range(j)]
        SETS=make_set_covering_problem(j,j,i)
        print("for density:",i,"and number of point:",j,"we have:\n\n")
        best_score,n_time=simulated_annealing(current_state,j)
        print("the best score with simulated annealing is: ",best_score,"and we call fitness function :",n_time,"\n")

        best_score,n_time =tabu_search(current_state,j)
        print("the best score with tabu search is: ",best_score,"and we call fitness function :",n_time,"\n")

        best_score,n_time =iterated_local_search(current_state,j)

        print("the best score with iterated local search is: ",best_score,"and we call fitness function :",n_time,"\n")



for density: 0.3 and number of point: 100 we have:


the best score with simulated annealing is:  (100, -8) and we call fitness function : 605 

the best score with tabu search is:  (100, -6) and we call fitness function : 19750 

the best score with iterated local search is:  (100, -8) and we call fitness function : 28 

for density: 0.3 and number of point: 1000 we have:


the best score with simulated annealing is:  (1000, -15) and we call fitness function : 605 

the best score with tabu search is:  (1000, -11) and we call fitness function : 19982 

the best score with iterated local search is:  (1000, -14) and we call fitness function : 56 

for density: 0.3 and number of point: 5000 we have:


the best score with simulated annealing is:  (5000, -24) and we call fitness function : 605 

the best score with tabu search is:  (5000, -15) and we call fitness function : 19988 

the best score with iterated local search is:  (5000, -21) and we call fitness function : 54 

for density: 0